### Импортируем библиотеки

In [1]:
import pandas as pd
import pandahouse as ph

### Задаем значения переменным

In [2]:
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'database':'simulator_20230320',
    'user':'student', 
    'password':'dpo_python_2020'
}

### Загружаем данные ленты новостей из БД

In [3]:
query = '''
SELECT user_id, 
         toDate(time) as event_date,
         gender,
         age,
         os,
         countIf(action, action='like') as likes,
         countIf(action, action='view') as views
  from simulator_20230320.feed_actions
  where toDate(time) = yesterday()
  group by user_id, event_date, gender, age, os
'''

In [4]:
df = ph.read_clickhouse(query, connection=connection)

In [5]:
df.head(10)

,user_id,event_date,gender,age,os,likes,views
0,128540,2023-04-09,1,23,Android,4,12
1,159526,2023-04-09,1,18,iOS,3,20
2,122270,2023-04-09,0,28,Android,0,9
3,66725,2023-04-09,1,20,Android,4,14
4,159590,2023-04-09,1,14,Android,2,14
5,65434,2023-04-09,0,24,iOS,2,10
6,128619,2023-04-09,0,26,iOS,14,64
7,157169,2023-04-09,0,38,Android,12,48
8,134587,2023-04-09,1,18,Android,12,66
9,71998,2023-04-09,0,17,iOS,7,46


### Загружаем данные мессенджера из БД

In [6]:
query2 = '''
SELECT event_date, user_id, gender, age, os, messages_sent, messages_received, users_sent, users_received
FROM
  (SELECT user_id, 
         toDate(time) as event_date,
         gender,
         age,
         os,
         count(user_id) as messages_sent,
         count(distinct reciever_id) as users_sent
  from simulator_20230320.message_actions
  where toDate(time) = yesterday()
  group by user_id, event_date, gender, age, os) as t1
FULL JOIN
  (SELECT reciever_id, 
         toDate(time) as event_date,
         count(reciever_id) as messages_received,
         count(distinct user_id) as users_received
  from simulator_20230320.message_actions 
  where toDate(time) = yesterday()
  group by reciever_id, event_date) as t2
ON t1.user_id = t2.reciever_id
'''

In [7]:
df2 = ph.read_clickhouse(query2, connection=connection)

In [8]:
df2.head(10)

,event_date,user_id,gender,age,os,messages_sent,messages_received,users_sent,users_received
0,2023-04-09,26363,1,25,iOS,5,0,5,0
1,2023-04-09,28593,0,24,Android,3,3,3,3
2,2023-04-09,107858,1,33,Android,2,4,2,4
3,2023-04-09,126512,0,17,Android,6,0,6,0
4,2023-04-09,118006,1,28,Android,5,0,5,0
5,2023-04-09,18242,1,17,iOS,5,13,4,12
6,2023-04-09,29995,1,16,iOS,4,0,4,0
7,2023-04-09,116006,1,16,iOS,3,0,3,0
8,2023-04-09,134208,0,23,Android,7,16,7,16
9,2023-04-09,8811,0,36,Android,3,3,2,2


### Объединяем таблицы

In [9]:
dfm = df.merge(df2, how='outer', on=['event_date', 'user_id', 'gender', 'age', 'os']).fillna(0)
dfm

### Считаем метрики по срезам

In [11]:
dimension_gender = dfm.groupby(['event_date', 'gender']).sum().reset_index()

In [12]:
dimension_gender['dimension']='gender'
dimension_gender.rename(columns={'gender':'dimension_value'}, inplace=True)
dimension_gender

,event_date,dimension_value,user_id,age,likes,views,messages_sent,messages_received,users_sent,users_received,dimension
0,1970-01-01,0,0,0.0,0.0,0.0,0.0,505.0,0.0,447.0,gender
1,2023-04-09,0,989692034,274937.0,53719.0,263320.0,5473.0,4991.0,4529.0,4108.0,gender
2,2023-04-09,1,1216214636,346144.0,66620.0,326434.0,7426.0,7403.0,6097.0,6071.0,gender


In [13]:
dimension_os = dfm.groupby(['event_date', 'os']).sum().reset_index()
dimension_os

,event_date,os,user_id,gender,age,likes,views,messages_sent,messages_received,users_sent,users_received
0,1970-01-01,,0,0.0,0.0,0.0,0.0,0.0,505.0,0.0,447.0
1,2023-04-09,Android,1437146873,8238.0,403961.0,78114.0,382613.0,8241.0,7632.0,6833.0,6313.0
2,2023-04-09,iOS,768759797,4454.0,217120.0,42225.0,207141.0,4658.0,4762.0,3793.0,3866.0


In [14]:
dimension_os['dimension']= 'os'
dimension_os.rename(columns={'os':'dimension_value'}, inplace=True)
dimension_os

,event_date,dimension_value,user_id,gender,age,likes,views,messages_sent,messages_received,users_sent,users_received,dimension
0,1970-01-01,,0,0.0,0.0,0.0,0.0,0.0,505.0,0.0,447.0,os
1,2023-04-09,Android,1437146873,8238.0,403961.0,78114.0,382613.0,8241.0,7632.0,6833.0,6313.0,os
2,2023-04-09,iOS,768759797,4454.0,217120.0,42225.0,207141.0,4658.0,4762.0,3793.0,3866.0,os


In [15]:
dimension_age = dfm.groupby(['event_date', 'age']).sum().reset_index()
dimension_age

,event_date,age,user_id,gender,likes,views,messages_sent,messages_received,users_sent,users_received
0,1970-01-01,0,0,0.0,0.0,0.0,0.0,505.0,0.0,447.0
1,2023-04-09,14,31786244,195.0,1555.0,7926.0,225.0,54.0,112.0,54.0
2,2023-04-09,15,72186853,427.0,3758.0,18704.0,423.0,134.0,285.0,131.0
3,2023-04-09,16,89519350,513.0,4920.0,23920.0,445.0,328.0,403.0,282.0
4,2023-04-09,17,105632866,603.0,5773.0,28229.0,560.0,528.0,543.0,463.0
...,...,...,...,...,...,...,...,...,...,...
62,2023-04-09,75,18666,0.0,3.0,23.0,0.0,0.0,0.0,0.0
63,2023-04-09,76,156662,1.0,1.0,11.0,0.0,0.0,0.0,0.0
64,2023-04-09,82,65425,0.0,7.0,45.0,0.0,0.0,0.0,0.0
65,2023-04-09,86,69550,0.0,12.0,56.0,0.0,0.0,0.0,0.0


In [16]:
dimension_age['dimension']= 'age'
dimension_age.rename(columns={'age':'dimension_value'}, inplace=True)
dimension_age

,event_date,dimension_value,user_id,gender,likes,views,messages_sent,messages_received,users_sent,users_received,dimension
0,1970-01-01,0,0,0.0,0.0,0.0,0.0,505.0,0.0,447.0,age
1,2023-04-09,14,31786244,195.0,1555.0,7926.0,225.0,54.0,112.0,54.0,age
2,2023-04-09,15,72186853,427.0,3758.0,18704.0,423.0,134.0,285.0,131.0,age
3,2023-04-09,16,89519350,513.0,4920.0,23920.0,445.0,328.0,403.0,282.0,age
4,2023-04-09,17,105632866,603.0,5773.0,28229.0,560.0,528.0,543.0,463.0,age
...,...,...,...,...,...,...,...,...,...,...,...
62,2023-04-09,75,18666,0.0,3.0,23.0,0.0,0.0,0.0,0.0,age
63,2023-04-09,76,156662,1.0,1.0,11.0,0.0,0.0,0.0,0.0,age
64,2023-04-09,82,65425,0.0,7.0,45.0,0.0,0.0,0.0,0.0,age
65,2023-04-09,86,69550,0.0,12.0,56.0,0.0,0.0,0.0,0.0,age


### Создаем итоговую таблицу

In [26]:
df_all = pd.concat([dimension_gender, dimension_os, dimension_age])
df_all = df_all[['event_date', 'dimension', 'dimension_value', 'views', 'likes', 'messages_received', 'messages_sent', 'users_received', 'users_sent']]
df_all[['views', 'likes', 'messages_received', 'messages_sent', 'users_received', 'users_sent']] = \
    df_all[['views', 'likes', 'messages_received', 'messages_sent', 'users_received', 'users_sent']].astype('int64')

In [27]:
df_all.dtypes

event_date           datetime64[ns]
dimension                    object
dimension_value              object
views                         int64
likes                         int64
messages_received             int64
messages_sent                 int64
users_received                int64
users_sent                    int64
dtype: object

### Записываем получившуюся таблицу в БД

In [23]:
query_create_table = '''
CREATE TABLE IF NOT EXISTS test.etl_k_shirochenkov (
    event_date      date,
    dimension       varchar(40),
    dimension_value varchar(40),
    views           integer,
    likes           integer,
    messages_received integer,
    messages_sent   integer,
    users_received  integer,
    users_sent      integer
)
ENGINE = Log()
'''

In [24]:
connection_test = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'database':'test',
    'user':'student-rw', 
    'password':'656e2b0c9c'
}

In [28]:
table_name = 'etl_k_shirochenkov'
ph.to_clickhouse(df_all, table_name, connection=connection_test, index=False)

73